# 최근 영업일 기준 데이터 받기

In [1]:
import requests as rq 
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.01.17


In [3]:
# regex
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20231121


# 한국거래소의 업종분류 현황 및 개별지표 크롤링

## 업종분류 현황 크롤링

In [4]:
# OTP를 받아오는 과정

import requests as rq
from io import BytesIO
import pandas as pd 

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId' : 'STK',        # STK는 코스피
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 헤더 부분에 레퍼러 추가 : 첫번째 URL에서 OTP를 부여받고, 이를 다시 두번째 URL에 제공하는 과정에서 레퍼러 없이 OTP를 전달하면 봇으로 인식해 데이터를 주지 않는다.
headers = {'Referer':  'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
# post() 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이 중에 텍스트에 해당하는 내용만 불러온다.
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

E3QYxRw+guOeN5kZgZ0Hk+vkNZV3XCEHnND8YyBRbNgRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15ecS9LieKO9XQUn8yJZ6qI3MtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [5]:
# OTP 제출 후 데이터 다운로드
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
# 받은 데이터의 content 부분을 ByteIO()를 이용해 바이너리 스트림 형태로 변환 후, read_csv() 함수로 데이터를 읽어 온다.
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4160,-5,-0.12,188251477440
1,006840,AK홀딩스,KOSPI,기타금융,17910,-70,-0.39,237263817510
2,027410,BGF,KOSPI,기타금융,3570,0,0.00,341708943870
3,282330,BGF리테일,KOSPI,유통업,135100,-300,-0.22,2335055700600
4,138930,BNK금융지주,KOSPI,기타금융,7130,30,0.42,2296490562940


In [6]:
# 코스닥 데이터 다운로드
gen_otp_ksq = {
    'mktId': 'KSQ',      # 코스닥 코드
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2310,-10,-0.43,112119643020
1,054620,APS,KOSDAQ,금융,6930,90,1.32,141331951530
2,265520,AP시스템,KOSDAQ,반도체,19700,-100,-0.51,301043993700
3,211270,AP위성,KOSDAQ,통신장비,14380,150,1.05,216883531520
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,4300,30,0.70,232343082500


In [7]:
# 코스피 데이터와 코스닥 데이터 결합
# concat으로 데이터 결합, 인덱스 초기화 및 인덱스 열 삭제
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
# 종목명에 공백이 있는 경우가 있으므로 strip() 메서드를 이용해 이를 제거
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
# 기준일 열 추가
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4160,-5,-0.12,188251477440,20231121
1,006840,AK홀딩스,KOSPI,기타금융,17910,-70,-0.39,237263817510,20231121
2,027410,BGF,KOSPI,기타금융,3570,0,0.00,341708943870,20231121
3,282330,BGF리테일,KOSPI,유통업,135100,-300,-0.22,2335055700600,20231121
4,138930,BNK금융지주,KOSPI,기타금융,7130,30,0.42,2296490562940,20231121


## 개별 종목 지표 크롤링

In [8]:
import requests as rq 
from io import BytesIO
import pandas as pd 

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

EmptyDataError: No columns to parse from file

# 데이터 정리하기

In [ ]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['제이알글로벌리츠', 'SK리츠', '씨케이에이치', '소마젠', '윙입푸드', '엑세스바이오', '프레스티지바이오파마', 'ESR켄달스퀘어리츠', 'NH올원리츠', '신한알파리츠', '맥쿼리인프라', '헝셩그룹', '오가닉티코스메틱', 'KB스타리츠', '한국패러랠', '코오롱티슈진', '모두투어리츠', '로스웰', '네오이뮨텍', '고스트스튜디오', '미래에셋글로벌리츠', '삼성FN리츠', 'GRT', 'SBI핀테크솔루션즈', '신한서부티엔디리츠', '롯데리츠', '이지스밸류리츠', '코람코더원리츠', '케이탑리츠', '한화리츠', '한국ANKOR유전', '맵스리얼티1', '컬러레이', '바다로19호', '이지스레지던스리츠', '엘브이엠씨홀딩스', '디앤디플랫폼리츠', '이리츠코크렙', '미래에셋맵스리츠', '골든센츄리', '잉글우드랩', '크리스탈신소재', '이스트아시아홀딩스', 'JTC', '글로벌에스엠', '마스턴프리미어리츠', '애머릿지', '에이리츠', '코람코라이프인프라리츠', 'NH프라임리츠']


In [ ]:
# 선박펀드, 광물펀드, 해외종목 등 일반적이지 않은 종목들이라 다음 두 데이터를 합쳐 준다.
kor_ticker = pd.merge(
    krx_sector,
    krx_ind,
    on=krx_sector.columns.intersection(
        krx_ind.columns
    ).to_list(), how='outer'
)

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4150,-35,-0.84,187798949850,20231117,201.0,20.65,612.0,6.78,8076.0,0.51,270.0,6.51
1,006840,AK홀딩스,KOSPI,기타금융,18000,60,0.33,238456098000,20231117,NaN,NaN,NaN,NaN,41948.0,0.43,200.0,1.11
2,027410,BGF,KOSPI,기타금융,3540,10,0.28,338837440140,20231117,247.0,14.33,NaN,NaN,16528.0,0.21,110.0,3.11
3,282330,BGF리테일,KOSPI,유통업,133500,-6100,-4.37,2307401451000,20231117,11203.0,11.92,12512.0,10.67,55724.0,2.40,4100.0,3.07
4,138930,BNK금융지주,KOSPI,기타금융,7070,0,0.00,2277165256660,20231117,2404.0,2.94,2440.0,2.90,30468.0,0.23,625.0,8.84


In [ ]:
# 마지막으로 일반적인 종목과 스펙, 우선주, 리츠, 기타 주식을 구분해주록 한다.

print(kor_ticker[kor_ticker['종목명'].str.contains('스펙|제[0-9]+호')]['종목명'].values)

['IBKS제19호스팩' 'IBKS제20호스팩' 'IBKS제21호스팩' 'IBKS제22호스팩' 'KB제25호스팩' 'KB제26호스팩'
 'KB제27호스팩' '대신밸런스제13호스팩' '대신밸런스제14호스팩' '대신밸런스제15호스팩' '대신밸런스제16호스팩'
 '비엔케이제1호스팩' '상상인제3호스팩' '상상인제4호스팩' '신한제10호스팩' '신한제11호스팩' '신한제8호스팩'
 '신한제9호스팩' '에스케이증권제10호스팩' '에스케이증권제8호스팩' '에스케이증권제9호스팩' '에이치엠씨제6호스팩'
 '유안타제10호스팩' '유안타제11호스팩' '유안타제12호스팩' '유안타제13호스팩' '유안타제14호스팩' '유안타제9호스팩'
 '케이비제21호스팩' '케이비제22호스팩' '키움제6호스팩' '키움제7호스팩' '키움제8호스팩' '하이제6호스팩' '하이제7호스팩'
 '하이제8호스팩' '한국제11호스팩' '한국제12호스팩' '한국제13호스팩' '한화플러스제2호스팩' '한화플러스제3호스팩'
 '한화플러스제4호스팩']


In [ ]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

['BYC우' 'CJ4우(전환)' 'CJ씨푸드1우' 'CJ우' 'CJ제일제당 우' 'DL우' 'DL이앤씨2우(전환)' 'DL이앤씨우'
 'GS우' 'JW중외제약2우B' 'JW중외제약우' 'LG생활건강우' 'LG우' 'LG전자우' 'LG화학우' 'LX하우시스우'
 'LX홀딩스1우' 'NH투자증권우' 'NPC우' 'S-Oil우' 'SK디스커버리우' 'SK우' 'SK이노베이션우' 'SK증권우'
 'SK케미칼우' '계양전기우' '금강공업우' '금호건설우' '금호석유우' '깨끗한나라우' '남선알미우' '남양유업우' '넥센우'
 '넥센타이어1우B' '노루페인트우' '노루홀딩스우' '녹십자홀딩스2우' '대교우B' '대덕1우' '대덕전자1우' '대상우'
 '대상홀딩스우' '대신증권2우B' '대신증권우' '대원전선우' '대한제당우' '대한항공우' '덕성우' '동부건설우' '동양2우B'
 '동양우' '동원시스템즈우' '두산2우B' '두산우' '두산퓨얼셀1우' '두산퓨얼셀2우B' '롯데지주우' '롯데칠성우'
 '미래에셋증권2우B' '미래에셋증권우' '부국증권우' '삼성SDI우' '삼성물산우B' '삼성전기우' '삼성전자우' '삼성화재우'
 '삼양사우' '삼양홀딩스우' '서울식품우' '성문전자우' '성신양회우' '세방우' '솔루스첨단소재1우' '솔루스첨단소재2우B'
 '신영증권우' '신풍제약우' '아모레G3우(전환)' '아모레G우' '아모레퍼시픽우' '유안타증권우' '유유제약1우'
 '유유제약2우B' '유한양행우' '유화증권우' '일양약품우' '진흥기업2우B' '진흥기업우B' '코리아써우' '코리아써키트2우B'
 '코오롱글로벌우' '코오롱모빌리티그룹우' '코오롱우' '코오롱인더우' '크라운제과우' '크라운해태홀딩스우' '태양금속우'
 '태영건설우' '티와이홀딩스우' '하이트진로2우B' '하이트진로홀딩스우' '한국금융지주우' '한양증권우' '한진칼우' '한화3우B'
 '한화갤러리아우' '한화솔루션우' '한화우' '한화투자증권우' '현대건설우' '현대차2우B' '현대차3우B' '현대차우

In [ ]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

['ESR켄달스퀘어리츠' 'KB스타리츠' 'NH올원리츠' 'NH프라임리츠' 'SK리츠' '디앤디플랫폼리츠' '롯데리츠'
 '마스턴프리미어리츠' '모두투어리츠' '미래에셋글로벌리츠' '미래에셋맵스리츠' '삼성FN리츠' '신한서부티엔디리츠' '신한알파리츠'
 '에이리츠' '이지스레지던스리츠' '이지스밸류리츠' '제이알글로벌리츠' '케이탑리츠' '코람코더원리츠' '코람코라이프인프라리츠'
 '한화리츠']


In [ ]:
# 해당 종목들을 구분하여 표기
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff), '기타', '보통주'))))

kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')        # 열이름 공백 제거
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})     # SQL에는 NaN이 입력되지 않으므로 None으로 변경
# kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4150,187798949850,20231117,201.0,612.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,18000,238456098000,20231117,None,None,41948.0,200.0,보통주
2,027410,BGF,KOSPI,3540,338837440140,20231117,247.0,None,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,133500,2307401451000,20231117,11203.0,12512.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,7070,2277165256660,20231117,2404.0,2440.0,30468.0,625.0,보통주


In [ ]:
import pymysql

con = pymysql.connect(user='root',
                      passwd='04250629',
                      host='127.0.0.1',
                      db='stock',
                      charset='utf8')

mycursor = con.cursor()
query = f""" 
INSERT INTO ticker_kr (code, company, market, close, market_cap, date, eps, forward_eps, bps, dividend, category)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
name=VALUES(name), market=VALUES(market), close=VALUES(close), market_cap=VALUES(market_cap), eps=VALUES(eps),
forward_eps=VALUES(forward_eps), bps=VALUES(bps), dividend=VALUES(dividend), category=VALUES(category);
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

# WICS 기준 섹터 정보 크롤링

In [ ]:
import json
import requests as rq 
import pandas as pd 
 
url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''    
data = rq.get(url).json()

type(data)

dict

In [ ]:
print(data.keys())

dict_keys(['info', 'list', 'sector', 'size'])


In [ ]:
# list에는 해당 섹터의 구성종목 정보가, sector에는 각종 섹터의 코드 정보가 포함
data['list'][0]

{'IDX_CD': 'G10',
 'IDX_NM_KOR': 'WICS 에너지',
 'ALL_MKT_VAL': 22138533,
 'CMP_CD': '096770',
 'CMP_KOR': 'SK이노베이션',
 'MKT_VAL': 7671564,
 'WGT': 34.65,
 'S_WGT': 34.65,
 'CAL_WGT': 1.0,
 'SEC_CD': 'G10',
 'SEC_NM_KOR': '에너지',
 'SEQ': 1,
 'TOP60': 3,
 'APT_SHR_CNT': 56367116}

In [ ]:
data['sector']

[{'SEC_CD': 'G25', 'SEC_NM_KOR': '경기관련소비재', 'SEC_RATE': 9.93, 'IDX_RATE': 0},
 {'SEC_CD': 'G35', 'SEC_NM_KOR': '건강관리', 'SEC_RATE': 9.08, 'IDX_RATE': 0},
 {'SEC_CD': 'G50', 'SEC_NM_KOR': '커뮤니케이션서비스', 'SEC_RATE': 6.29, 'IDX_RATE': 0},
 {'SEC_CD': 'G40', 'SEC_NM_KOR': '금융', 'SEC_RATE': 7.96, 'IDX_RATE': 0},
 {'SEC_CD': 'G10', 'SEC_NM_KOR': '에너지', 'SEC_RATE': 1.85, 'IDX_RATE': 100.0},
 {'SEC_CD': 'G20', 'SEC_NM_KOR': '산업재', 'SEC_RATE': 12.02, 'IDX_RATE': 0},
 {'SEC_CD': 'G55', 'SEC_NM_KOR': '유틸리티', 'SEC_RATE': 0.93, 'IDX_RATE': 0},
 {'SEC_CD': 'G30', 'SEC_NM_KOR': '필수소비재', 'SEC_RATE': 2.25, 'IDX_RATE': 0},
 {'SEC_CD': 'G15', 'SEC_NM_KOR': '소재', 'SEC_RATE': 9.39, 'IDX_RATE': 0},
 {'SEC_CD': 'G45', 'SEC_NM_KOR': 'IT', 'SEC_RATE': 40.31, 'IDX_RATE': 0}]

In [ ]:
# list -> dataframe

data_pd = pd.json_normalize(data['list'])
data_pd.head()

,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,22138533,096770,SK이노베이션,7671564,34.65,34.65,1.0,G10,에너지,1,3,56367116
1,G10,WICS 에너지,22138533,009830,한화솔루션,3438280,15.53,50.18,1.0,G10,에너지,2,3,108292298
2,G10,WICS 에너지,22138533,010950,S-Oil,2824252,12.76,62.94,1.0,G10,에너지,3,3,41655633
3,G10,WICS 에너지,22138533,267250,HD현대,2623202,11.85,74.79,1.0,G10,에너지,4,3,44236128
4,G10,WICS 에너지,22138533,078930,GS,2092919,9.45,84.24,1.0,G10,에너지,5,3,49245150


In [ ]:
import time
import json
import requests as rq 
import pandas as pd 
from tqdm import tqdm

sector_code = [
    'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15','G45'
]

data_sector = []

for i in tqdm(sector_code):
    url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''    
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])

    data_sector.append(data_pd)

    time.sleep(2)

100%|██████████| 10/10 [00:22<00:00,  2.24s/it]


In [ ]:
kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])
kor_sector.head()

,IDX_CD,CMP_CD,CMP_KOR,SEC_NM_KOR,기준일
0,G25,005380,현대차,경기관련소비재,2023-11-17
1,G25,000270,기아,경기관련소비재,2023-11-17
2,G25,012330,현대모비스,경기관련소비재,2023-11-17
3,G25,090430,아모레퍼시픽,경기관련소비재,2023-11-17
4,G25,161390,한국타이어앤테크놀로지,경기관련소비재,2023-11-17


In [ ]:
import pymysql

con = pymysql.connect(user='root',
                      passwd='04250629',
                      host='127.0.0.1',
                      db='stock',
                      charset='utf8')

mycursor = con.cursor()
query = f"""
INSERT INTO sector_kr (index_code, company_code, company, sec_nm_kr, date)
VALUES (%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
index_code=VALUES(index_code), company_code=VALUES(company_code), company=VALUES(company), sec_nm_kr=VALUES(sec_nm_kr)
"""

args = kor_sector.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

# 수정 주가 크롤링

## 개별종목 주가 크롤링

In [ ]:
# geting tickers from database

from sqlalchemy import create_engine
import pandas as pd

user = 'root'
pw = '04250629'
host = '127.0.0.1'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
query = """ 
SELECT * FROM ticker_kr
WHERE date = (SELECT MAX(date) FROM ticker_kr)
    AND category = '보통주';
"""

ticker_list = pd.read_sql(query, con=engine)
engine.dispose()

ticker_list.head()

,code,company,market,close,market_cap,date,eps,forward_eps,bps,dividend,category
0,000020,동화약품,KOSPI,9500.0,2.653490e+11,2023-11-17,736.0,NaN,13165.0,180.0,보통주
1,000040,KR모터스,KOSPI,527.0,5.066500e+10,2023-11-17,NaN,NaN,345.0,0.0,보통주
2,000050,경방,KOSPI,8520.0,2.335780e+11,2023-11-17,177.0,NaN,30304.0,125.0,보통주
3,000070,삼양홀딩스,KOSPI,70300.0,6.020680e+11,2023-11-17,9173.0,NaN,240995.0,3500.0,보통주
4,000080,하이트진로,KOSPI,23250.0,1.630610e+12,2023-11-17,1250.0,1203.0,16906.0,950.0,보통주


In [ ]:
# 주가 데이터 크롤링
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date

i = 0
ticker = ticker_list['code'][i]
fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d")
to = (date.today()).strftime("%Y%m%d")

url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
&startTime={fr}&endTime={to}&timeframe=day'''

data = rq.get(url).content
data_price = pd.read_csv(BytesIO(data))
data_price

,[['날짜','시가','고가','저가','종가','거래량','외국인소진율'],Unnamed: 7
0,"[""20181123""",9590.0,9630.0,9410.0,9480.0,40882.0,8.5],NaN
1,"[""20181126""",9580.0,9580.0,9390.0,9400.0,41585.0,8.5],NaN
2,"[""20181127""",9450.0,9510.0,9410.0,9430.0,21198.0,8.5],NaN
3,"[""20181128""",9480.0,9490.0,9150.0,9320.0,138261.0,8.5],NaN
4,"[""20181129""",9410.0,9410.0,9260.0,9360.0,47101.0,8.47],NaN
...,...,...,...,...,...,...,...,...
1229,"[""20231117""",9630.0,9630.0,9500.0,9500.0,41158.0,4.32],NaN
1230,"[""20231120""",9500.0,9600.0,9500.0,9600.0,23277.0,4.33],NaN
1231,"[""20231121""",9710.0,9720.0,9600.0,9680.0,62806.0,4.37],NaN
1232,"[""20231122""",9670.0,9690.0,9600.0,9650.0,46025.0,4.37],NaN


In [ ]:
# clensing data
import re

price = data_price.iloc[:, 0:6]
price.columns = ['date', 'open', 'high', 'low', 'close', 'volume'] 
price = price.dropna()
price['date'] = price['date'].str.extract('(\d+)')  # 정규식을 이용해 열에서 숫자만 추출
price['date'] = pd.to_datetime(price['date'])
price['code'] = ticker

price.head()

,date,open,high,low,close,volume,code
0,2018-11-23,9590.0,9630.0,9410.0,9480.0,40882.0,000020
1,2018-11-26,9580.0,9580.0,9390.0,9400.0,41585.0,000020
2,2018-11-27,9450.0,9510.0,9410.0,9430.0,21198.0,000020
3,2018-11-28,9480.0,9490.0,9150.0,9320.0,138261.0,000020
4,2018-11-29,9410.0,9410.0,9260.0,9360.0,47101.0,000020


## 전체 종목 주가 크롤링

In [9]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import requests as rq
import time
from tqdm import tqdm
from io import BytesIO

# DB 연결
user = 'root'
pw = '04250629'
host = '127.0.0.1'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
con = pymysql.connect(
    user=user,
    passwd=pw,
    host=host,
    db=db,
    charset='utf8'   
)
mycursor = con.cursor()

In [10]:
# 티커리스트 불러오기
ticker_list = pd.read_sql(
    """ 
    SELECT * FROM ticker_kr 
    WHERE date = (SELECT MAX(date) FROM ticker_kr)
        AND category='보통주';
    """, con=engine
)
ticker_list

,company_code,company,market,close,market_cap,date,eps,forward_eps,bps,dividend,category
0,000020,동화약품,KOSPI,9500.0,2.653490e+11,2023-11-17,736.0,NaN,13165.0,180.0,보통주
1,000040,KR모터스,KOSPI,527.0,5.066500e+10,2023-11-17,NaN,NaN,345.0,0.0,보통주
2,000050,경방,KOSPI,8520.0,2.335780e+11,2023-11-17,177.0,NaN,30304.0,125.0,보통주
3,000070,삼양홀딩스,KOSPI,70300.0,6.020680e+11,2023-11-17,9173.0,NaN,240995.0,3500.0,보통주
4,000080,하이트진로,KOSPI,23250.0,1.630610e+12,2023-11-17,1250.0,1203.0,16906.0,950.0,보통주
...,...,...,...,...,...,...,...,...,...,...,...
2390,457190,이수스페셜티케미컬,KOSPI,151600.0,8.487740e+11,2023-11-17,NaN,NaN,NaN,0.0,보통주
2391,460850,동국씨엠,KOSPI,7050.0,2.107860e+11,2023-11-17,NaN,1483.0,NaN,0.0,보통주
2392,460860,동국제강,KOSPI,10460.0,5.189000e+11,2023-11-17,NaN,7958.0,NaN,0.0,보통주
2393,462520,조선내화,KOSPI,25650.0,3.040850e+11,2023-11-17,NaN,NaN,NaN,0.0,보통주


In [11]:
# query
query = """ 
INSERT INTO price_kr (date, open, high, low, close, volume, company_code)
VALUES (%s,%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
open=VALUES(open), high=VALUES(high), low=VALUES(low), close=VALUES(close), volume=VALUES(volume);
"""

# 오류 발생시 저장할 리스트
error_list = []

# 전 종목 주가 다운로드 및 저장
for i in tqdm(range(0, len(ticker_list))):
    
    # pick a ticker
    ticker = ticker_list['company_code'][i]
    
    # start date and end date
    fr = (date.today() + relativedelta(years=-30)).strftime('%Y%m%d')
    to = (date.today()).strftime('%Y%m%d')
    
    # 오류 발생 시 무시하고 다음루프 진행
    try:
        
        # url
        url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
        &startTime={fr}&endTime={to}&timeframe=day'''
        
        # download data
        data = rq.get(url).content
        data_price = pd.read_csv(BytesIO(data))
        
        # data clensing
        price = data_price.iloc[:, 0:6]
        price.columns = ['date', 'open', 'high', 'low', 'close', 'volume'] 
        price = price.dropna()
        price['date'] = price['date'].str.extract('(\d+)')  # 정규식을 이용해 열에서 숫자만 추출
        price['date'] = pd.to_datetime(price['date'])
        price['code'] = ticker
        
        # insert db
        args = price.values.tolist()
        mycursor.executemany(query, args)
        con.commit()
    
    except:
        print(ticker)
        error_list.append(ticker)
        
    # time sleep
    time.sleep(2)
    
# db 연결 종료
engine.dispose()
con.close()
        

  0%|          | 0/2395 [00:00<?, ?it/s]

100%|██████████| 2395/2395 [1:30:19<00:00,  2.26s/it]


# 재무제표 크롤링

재무제표는 여러 웹사이트에서 구할 수 있으며, 국내 데이터 제공업체인 FnGiude에서 운영하는 Compnay Guide 웹사이트에서 손쉽게 구할 수 있다.

[http://comp.fnguide.com/](http://comp.fnguide.com/)

## 재무제표 다운로드

개별종목의 재무제표 탭을 선택하면, 포괄손익계산서, 재무상태표, 현금흐름표 항목이 있으며, 티커에 해당하는 A005930 뒤의 주소는 불필요한 내용이므로, 이를 제거한 주소로 접속한다. A뒤의 6자리 티커만 변경한다면 해당 종목의 재무제표 페이지로 이동하게 된다.

[http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930](http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930)

In [43]:
# 삼성전자 종목의 페이지 내용 불러오기
from sqlalchemy import create_engine
import pandas as pd 

# DB 연결
user = 'root'
pw = '04250629'
host = '127.0.0.1'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
query = """ 
SELECT * FROM ticker_kr
WHERE date = {SELECT MAX(date) FROM ticker_kr}
    AND category = '보통주';
"""
engine.dispose()

i = 0
ticker = ticker_list['company_code'][i]

url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}"
data = pd.read_html(url, displayed_only=False)

# data

[item.head(3) for item in data]

[  IFRS(연결)  2020/12  2021/12  2022/12  2023/09    전년동기 전년동기(%)
 0      매출액   2721.0   2930.0   3404.0   2769.0  2560.0     8.2
 1     매출원가   1334.0   1437.0   1594.0   1330.0  1209.0    10.1
 2    매출총이익   1387.0   1493.0   1810.0   1438.0  1351.0     6.5,
   IFRS(연결)  2022/12  2023/03  2023/06  2023/09   전년동기 전년동기(%)
 0      매출액    845.0    994.0    900.0    875.0  835.0     4.8
 1     매출원가    386.0    468.0    438.0    424.0  398.0     6.5
 2    매출총이익    459.0    526.0    462.0    450.0  436.0     3.2,
              IFRS(연결)  2020/12  2021/12  2022/12  2023/09
 0                  자산   4338.0   4478.0   4611.0   4902.0
 1  유동자산계산에 참여한 계정 펼치기   2227.0   2202.0   2275.0   2346.0
 2                재고자산    395.0    362.0    468.0    547.0,
              IFRS(연결)  2022/12  2023/03  2023/06  2023/09
 0                  자산   4611.0   4770.0   4818.0   4902.0
 1  유동자산계산에 참여한 계정 펼치기   2275.0   2357.0   2317.0   2346.0
 2                재고자산    468.0    494.0    549.0    547.0,
          IFRS(연

In [44]:
print(
    data[0].columns.to_list(), '\n',
    data[2].columns.tolist(), '\n',
    data[4].columns.tolist()
)

['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/09', '전년동기', '전년동기(%)'] 
 ['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/09'] 
 ['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/09']


In [45]:
data_fs_y = pd.concat(
    [data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2], data[4]]
)
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})

data_fs_y.head()

,계정,2020/12,2021/12,2022/12,2023/09
0,매출액,2721.0,2930.0,3404.0,2769.0
1,매출원가,1334.0,1437.0,1594.0,1330.0
2,매출총이익,1387.0,1493.0,1810.0,1438.0
3,판매비와관리비계산에 참여한 계정 펼치기,1155.0,1269.0,1511.0,1235.0
4,인건비,415.0,468.0,489.0,455.0


In [46]:
# data crawling
# 결산월 추출하기
import requests as rq 
from bs4 import BeautifulSoup
import re

page_data = rq.get(url)
page_data_html = BeautifulSoup(page_data.content, 'html.parser')

fiscal_data = page_data_html.select('div.corp_group1 > h2')
fiscal_data_text = fiscal_data[1].text
fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)

print(fiscal_data_text)

['12']


In [47]:
# 재무제표에 해당하는 열만 선택
data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') |
                          (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
data_fs_y.head()

,계정,2020/12,2021/12,2022/12
0,매출액,2721.0,2930.0,3404.0
1,매출원가,1334.0,1437.0,1594.0
2,매출총이익,1387.0,1493.0,1810.0
3,판매비와관리비계산에 참여한 계정 펼치기,1155.0,1269.0,1511.0
4,인건비,415.0,468.0,489.0


In [48]:
# data clensing
data_fs_y[data_fs_y.loc[:, ~data_fs_y.columns.isin(['계정'])].isna().all(
    axis=1)].head()


,계정,2020/12,2021/12,2022/12
10,기타원가성비용,NaN,NaN,NaN
18,대손충당금환입액,NaN,NaN,NaN
19,매출채권처분이익,NaN,NaN,NaN
20,당기손익-공정가치측정 금융자산관련이익,NaN,NaN,NaN
23,금융자산손상차손환입,NaN,NaN,NaN


In [49]:
data_fs_y['계정'].value_counts(ascending=False).head()

계정
기타          4
배당금수익       3
파생상품이익      3
이자수익        3
법인세납부(-)    3
Name: count, dtype: int64

In [63]:
def clean_fs(df, ticker, frequency):
    """ 
    입력값으로 데이터프레임, 티커, 공시구분(연간/분기)가 필요
    """
    
    # 모든 연도의 데이터가 NaN인 항목 제외
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)] # ~ : not
    # 계정명이 중복인 경우 drop_duplicate() 함수를 이용해 첫번째에 위치하는 데이터만 남김
    df = df.drop_duplicates(['계정'], keep='first')
    # melt() 함수를 이용해 열로 긴 데이터를 행으로 긴 데이터로 변경한다.
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    # 계정값이 없는 항목 제외
    df = df[~pd.isnull(df['값'])]
    # [계산에 참여한 계정 펼치기]라는 글자는 페이지의 [+]에 해당하는 부분이므로 replace() 메서드를 통해 제거
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    # to_datetime() 메서드를 통해 기준일을 'yyyy-mm' 형태로 바꾼 후, MonthEnd()를 통해 월말에 해당하는 일을 불러옴
    df['기준일'] = pd.to_datetime(df['기준일'], format='%Y/%m') + pd.tseries.offsets.MonthEnd()
    # '종목코드' 열에 티커 입력
    df['종목코드'] = ticker
    # '공시구분' 열에는 연간 혹은 분기에 해당하는 값을 입력
    df['공시구분'] = frequency
    
    return df

In [64]:
# 함수 적용
data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')
data_fs_y_clean

,계정,기준일,값,종목코드,공시구분
0,매출액,2020-12-31,2721.0,000020,y
1,매출원가,2020-12-31,1334.0,000020,y
2,매출총이익,2020-12-31,1387.0,000020,y
3,판매비와관리비,2020-12-31,1155.0,000020,y
4,인건비,2020-12-31,415.0,000020,y
...,...,...,...,...,...
400,배당금지급(-),2022-12-31,-50.0,000020,y
401,환율변동효과,2022-12-31,-1.0,000020,y
402,현금및현금성자산의증가,2022-12-31,-261.0,000020,y
403,기초현금및현금성자산,2022-12-31,605.0,000020,y


In [66]:
# 분기 데이터 클렌딩
data_fs_q = pd.concat(
    [data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3], data[5]]
)
data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

data_fs_q_clean.head()

,계정,기준일,값,종목코드,공시구분
0,매출액,2022-12-31,845.0,000020,q
1,매출원가,2022-12-31,386.0,000020,q
2,매출총이익,2022-12-31,459.0,000020,q
3,판매비와관리비,2022-12-31,426.0,000020,q
4,인건비,2022-12-31,81.0,000020,q


In [67]:
# 분기 데이터느 결산월에 해당하는 부분을 선택할 필요가 없으며, 이를 제외하고는 모든 과정이 연간 재무제표의 항목과 동일
data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
data_fs_bind

,계정,기준일,값,종목코드,공시구분
0,매출액,2020-12-31,2721.0,000020,y
1,매출원가,2020-12-31,1334.0,000020,y
2,매출총이익,2020-12-31,1387.0,000020,y
3,판매비와관리비,2020-12-31,1155.0,000020,y
4,인건비,2020-12-31,415.0,000020,y
...,...,...,...,...,...
525,기타금융부채의감소,2023-09-30,7.0,000020,q
528,환율변동효과,2023-09-30,-1.0,000020,q
529,현금및현금성자산의증가,2023-09-30,137.0,000020,q
530,기초현금및현금성자산,2023-09-30,810.0,000020,q


# 전종목 재무제표 크롤링

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
import time

# DB 연결
user = 'root'
pw = '04250629'
host = '127.0.0.1'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
con = pymysql.connect(
    user=user,
    passwd=pw,
    host=host,
    db=db,
    charset='utf8'   
)
mycursor = con.cursor()

In [2]:
# 티커리스트 불러오기
ticker_list = pd.read_sql("""
SELECT * FROM ticker_kr
WHERE date = (SELECT MAX(date) FROM ticker_kr)
    AND category = '보통주';
""", con=engine)

# DB query
query = """ 
INSERT INTO fs_kr (account, date, value, company_code, frequency)
VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
value=VALUES(value)
"""

# 오류 발생시 저장할 리스트 생성
error_list = []

# 재무베표 클렌징 함수
def clean_fs(df, ticker, frequency):
    
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    df['기준일'] = pd.to_datetime(df['기준일'], format='%Y/%m') + pd.tseries.offsets.MonthEnd()
    df['종목코드'] = ticker
    df['공시구분'] = frequency
    
    return df

In [4]:
# for loop
for i in tqdm(range(0, len(ticker_list))):
    # select ticker
    ticker = ticker_list['company_code'][i]
    
    try:
        # url 
        url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
        
        # get data
        data = pd.read_html(url, displayed_only=False)
        
        # yearly data
        data_fs_y = pd.concat([
            data[0].iloc[:, ~data[0].columns.str.contains('전년공기')], 
            data[2], data[4]
        ])
        data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})
        
        # 결산년 찾기
        page_data = rq.get(url)
        page_data_html = BeautifulSoup(page_data.content, 'html.parser')
        
        fiscal_data = page_data_html.select('div.corp_group1 > h2')
        fiscal_data_text = fiscal_data[1].text
        fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)
        
        # 결산년에 해당하는 계정만 남기기
        data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') | (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
        
        # 클렌징
        data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')
        
        # quarterly data
        data_fs_q = pd.concat([
            data[1].iloc[:, ~ data[1].columns.str.contains('전년동기')],
            data[3], data[5]
        ])
        data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
        data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')
        
        # concat
        data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
        
        # insert into db
        args = data_fs_bind.values.tolist()
        mycursor.executemany(query, args)
        con.commit()
    
    except:
        # 오류 발생시 해당 종목명을 저장하고 다음 루프로 이동
        print(ticker)
        error_list.append(ticker)
        
    # time sleep
    time.sleep(2)
    
# DB 연결 종료
engine.dispose()
con.close()
        

100%|██████████| 2395/2395 [1:49:01<00:00,  2.73s/it]  


# 가치지표 계산하기

In [6]:
from sqlalchemy import create_engine
import pandas as pd

# connect db
user = 'root'
pw = '04250629'
host = '127.0.0.1'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')

# ticker list
ticekr_list = pd.read_sql("""
SELECT * FROM ticker_kr
WHERE date = (SELECT MAX(date) FROM ticker_kr)
    AND category = '보통주';
""", con=engine)

# 삼성전자 분기 재무제표
sample_fs = pd.read_sql("""
SELECT * FROM fs_kr
WHERE frequency='q'
AND company_code = '005930'
AND account in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
""", con=engine)

engine.dispose()

sample_fs

,account,date,value,company_code,frequency
0,당기순이익,2022-12-31,238414.0,005930,q
1,당기순이익,2023-03-31,15746.0,005930,q
2,당기순이익,2023-06-30,17236.0,005930,q
3,당기순이익,2023-09-30,58442.0,005930,q
4,매출액,2022-12-31,704646.0,005930,q
5,매출액,2023-03-31,637454.0,005930,q
6,매출액,2023-06-30,600055.0,005930,q
7,매출액,2023-09-30,674047.0,005930,q
8,영업활동으로인한현금흐름,2022-12-31,186129.0,005930,q
9,영업활동으로인한현금흐름,2023-03-31,62918.0,005930,q


In [9]:
# 종목코드, 계정, 기준일 순으로 정렬
sample_fs = sample_fs.sort_values(['company_code', 'account', 'date'])
sample_fs

,account,date,value,company_code,frequency
0,당기순이익,2022-12-31,238414.0,005930,q
1,당기순이익,2023-03-31,15746.0,005930,q
2,당기순이익,2023-06-30,17236.0,005930,q
3,당기순이익,2023-09-30,58442.0,005930,q
4,매출액,2022-12-31,704646.0,005930,q
5,매출액,2023-03-31,637454.0,005930,q
6,매출액,2023-06-30,600055.0,005930,q
7,매출액,2023-09-30,674047.0,005930,q
8,영업활동으로인한현금흐름,2022-12-31,186129.0,005930,q
9,영업활동으로인한현금흐름,2023-03-31,62918.0,005930,q


In [10]:
# 종목코드와 계정을 기준으로 groupby() 함수를 통해 그룹으로 묶음
# as_index=False를 통해 그룹 라벨을 인덱스로 사용하지 않음
# rolling() 메서드를 통해 4개 기간씩 합계를 구하며, min_periods 인자를 통해 데이터가 최소 4개 있는 경우에만 값을 구한다.
sample_fs['ttm'] = sample_fs.groupby(
    ['company_code', 'account'], as_index=False
)['value'].rolling(window=4, min_periods=4).sum()['value']
sample_fs

,account,date,value,company_code,frequency,ttm
0,당기순이익,2022-12-31,238414.0,005930,q,NaN
1,당기순이익,2023-03-31,15746.0,005930,q,NaN
2,당기순이익,2023-06-30,17236.0,005930,q,NaN
3,당기순이익,2023-09-30,58442.0,005930,q,329838.0
4,매출액,2022-12-31,704646.0,005930,q,NaN
5,매출액,2023-03-31,637454.0,005930,q,NaN
6,매출액,2023-06-30,600055.0,005930,q,NaN
7,매출액,2023-09-30,674047.0,005930,q,2616202.0
8,영업활동으로인한현금흐름,2022-12-31,186129.0,005930,q,NaN
9,영업활동으로인한현금흐름,2023-03-31,62918.0,005930,q,NaN


In [20]:
import numpy as np

# '자본' 항목은 재무상태표에 해당하는 항목이므로 합이 아닌 4로 나누어 평균을 구하며, 타 항목은 4분기 기준 합을 그대로 사용
sample_fs['ttm'] = np.where(sample_fs['account'] == '자본', sample_fs['ttm'] / 4, sample_fs['ttm'])
# 계정과 종목코드별 그룹을 나눈 후 tail(1) 함수를 통해 가장 최근 데이터만 선택
sample_fs = sample_fs.groupby(['account', 'company_code']).tail(1)

In [22]:
# 시가총액
sample_fs_merge = sample_fs[['account', 'company_code', 'ttm']].merge(
    ticekr_list[['company_code', 'market_cap', 'date']],
    on='company_code'
)
# 재무제표 데이터의 경우 단위가 억원인 반면, 시가총액은 원이므로, 시가총액을 억으로 나눠 단위를 맞춰준다.
sample_fs_merge['market_cap'] = sample_fs_merge['market_cap'] / 100000000
sample_fs_merge.head()

,account,company_code,ttm,market_cap,date
0,당기순이익,005930,329838.00,4328090.0,2023-11-17
1,매출액,005930,2616202.00,4328090.0,2023-11-17
2,영업활동으로인한현금흐름,005930,428051.00,4328090.0,2023-11-17
3,자본,005930,898386.25,4328090.0,2023-11-17


In [23]:
# 분자(시가총액)을 분보(TTM 데이터)로 나누어 가치지표 계산 후, 각 지표명을 입력
sample_fs_merge['value'] = sample_fs_merge['market_cap'] / sample_fs_merge['ttm']
sample_fs_merge['ratio'] = np.where(
    sample_fs_merge['account'] == '매출액', 'PSR',
    np.where(
        sample_fs_merge['account'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(sample_fs_merge['account'] == '자본', 'PBR',
                 np.where(sample_fs_merge['account'] == '당기순이익', 'PER', None))
    )
)
sample_fs_merge

,account,company_code,ttm,market_cap,date,value,ratio
0,당기순이익,005930,329838.00,4328090.0,2023-11-17,13.121866,PER
1,매출액,005930,2616202.00,4328090.0,2023-11-17,1.654341,PSR
2,영업활동으로인한현금흐름,005930,428051.00,4328090.0,2023-11-17,10.111155,PCR
3,자본,005930,898386.25,4328090.0,2023-11-17,4.817627,PBR


In [24]:
# 배당수익률
ticker_list_sample = ticker_list[ticker_list['company_code'] == '005930'].copy()
ticker_list_sample['DY'] = ticker_list_sample['dividend'] / ticker_list_sample['close']

ticker_list_sample.head()

,company_code,company,market,close,market_cap,date,eps,forward_eps,bps,dividend,category,DY
259,005930,삼성전자,KOSPI,72500.0,4.328090e+14,2023-11-17,8057.0,4019.0,57822.0,1444.0,보통주,0.019917


## 전 종목 가치지표 계산

In [25]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

# DB 연결
user = 'root'
pw = '04250629'
host = '127.0.0.1'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
con = pymysql.connect(
    user=user,
    passwd=pw,
    host=host,
    db=db,
    charset='utf8'   
)
mycursor = con.cursor()

In [29]:
# quaterly financial statements
kor_fs = pd.read_sql("""
SELECT * FROM fs_kr
WHERE frequency = 'q' 
    AND account IN ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
""", con=engine)

# ticker list
ticekr_list = pd.read_sql("""
SELECT * FROM ticker_kr
WHERE date = (SELECT MAX(date) FROM ticker_kr)
    AND category = '보통주';
""", con=engine)

engine.dispose()

In [31]:
# TTM calculation
kor_fs = kor_fs.sort_values(['company_code', 'account', 'date'])
kor_fs['ttm'] = kor_fs.groupby(['company_code', 'account'], as_index=False)['value'].rolling(
    window=4, min_periods=4
).sum()['value']

# 자본의 평균으로 대체
kor_fs['ttm'] = np.where(kor_fs['account'] == '자본', kor_fs['ttm'] / 4, kor_fs['ttm'])
kor_fs = kor_fs.groupby(['account', 'company_code']).tail(1)

In [34]:
# 티커리스트와 시가총액 데이터를 이용해 가치지표 계산'
kor_fs_merge = kor_fs[['account', 'company_code', 'ttm']].merge(
    ticekr_list[['company_code', 'market_cap', 'date']],
    on='company_code'
)
kor_fs_merge['market_cap'] = kor_fs_merge['market_cap'] / 100000000

kor_fs_merge['value'] = kor_fs_merge['market_cap'] / kor_fs_merge['ttm']
kor_fs_merge['ratio'] = np.where(
    kor_fs_merge['account'] =='매출액', 'PSR',
    np.where(
        kor_fs_merge['account'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(kor_fs_merge['account'] == '자본', 'PBR',
                 np.where(kor_fs_merge['account'] == '당기순이익', 'PER', None))
    )
)

# kor_fs_merge.rename(columns={'value': '값'}, inplace=True)
kor_fs_merge = kor_fs_merge[['company_code', 'date', 'ratio', 'value']]
kor_fs_merge = kor_fs_merge.replace([np.inf, -np.inf, np.nan], None)

kor_fs_merge.head(4)

,company_code,date,ratio,value
0,000020,2023-11-17,PER,9.087295
1,000020,2023-11-17,PSR,0.734225
2,000020,2023-11-17,PCR,7.249973
3,000020,2023-11-17,PBR,0.679946


In [36]:
# inser into database
query = """ 
INSERT INTO value_kr (company_code, date, ratio, value)
VALUES (%s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
value = VALUES(value)
"""

arg_fs = kor_fs_merge.values.tolist()
mycursor.executemany(query, arg_fs)
con.commit()

In [38]:
# 배당수익률
ticker_list['value'] = ticker_list['dividend'] / ticker_list['close']
ticker_list['value'] = ticker_list['value'].round(4)
ticker_list['ratio'] = 'DY'
dy_list = ticker_list[['company_code', 'date', 'ratio', 'value']]
dy_list = dy_list.replace([np.inf, -np.inf, np.nan], None)
dy_list = dy_list[dy_list['value'] != 0]

dy_list.head()

,company_code,date,ratio,value
0,000020,2023-11-17,DY,0.0189
2,000050,2023-11-17,DY,0.0147
3,000070,2023-11-17,DY,0.0498
4,000080,2023-11-17,DY,0.0409
5,000100,2023-11-17,DY,0.0065


In [39]:
# insert into database
args_dy = dy_list.values.tolist()
mycursor.executemany(query, args_dy)
con.commit()

engine.dispose()
con.close()